# Análise Cripto Moedas

### Imports

In [37]:
import requests
from binance.spot import Spot 
from binance.error import ClientError
import numpy as np
import pandas as pd
from pandas_datareader import data

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
from datetime import datetime

from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import SMAIndicator
from ta.trend import MACD
from ta.volume import OnBalanceVolumeIndicator

## Escolhendo os Simbolos

In [71]:
spot_client = Spot()
result = spot_client.exchange_info()
result


import json
with open('data.json', 'w') as f:
    json.dump(result, f)
    
order_types = [
        "LIMIT",
        "LIMIT_MAKER",
        "MARKET",
        "STOP_LOSS_LIMIT",
        "TAKE_PROFIT_LIMIT"
      ]
    
## ESTUDAR O QUE É USDT, BTC    
symbols = [symbol["symbol"] for symbol in result["symbols"] if symbol["status"] == "TRADING" and symbol["quoteAsset"] == "USDT" and all(ot in order_types for ot in symbol["orderTypes"])]
len(symbols)
symbols

['BTCUSDT',
 'ETHUSDT',
 'BNBUSDT',
 'NEOUSDT',
 'LTCUSDT',
 'QTUMUSDT',
 'ADAUSDT',
 'XRPUSDT',
 'EOSUSDT',
 'TUSDUSDT',
 'IOTAUSDT',
 'XLMUSDT',
 'ONTUSDT',
 'TRXUSDT',
 'ETCUSDT',
 'ICXUSDT',
 'NULSUSDT',
 'VETUSDT',
 'USDCUSDT',
 'LINKUSDT',
 'WAVESUSDT',
 'BTTUSDT',
 'ONGUSDT',
 'HOTUSDT',
 'ZILUSDT',
 'ZRXUSDT',
 'FETUSDT',
 'BATUSDT',
 'XMRUSDT',
 'ZECUSDT',
 'IOSTUSDT',
 'CELRUSDT',
 'DASHUSDT',
 'NANOUSDT',
 'OMGUSDT',
 'THETAUSDT',
 'ENJUSDT',
 'MITHUSDT',
 'MATICUSDT',
 'ATOMUSDT',
 'TFUELUSDT',
 'ONEUSDT',
 'FTMUSDT',
 'ALGOUSDT',
 'GTOUSDT',
 'DOGEUSDT',
 'DUSKUSDT',
 'ANKRUSDT',
 'WINUSDT',
 'COSUSDT',
 'COCOSUSDT',
 'MTLUSDT',
 'TOMOUSDT',
 'PERLUSDT',
 'DENTUSDT',
 'MFTUSDT',
 'KEYUSDT',
 'DOCKUSDT',
 'WANUSDT',
 'FUNUSDT',
 'CVCUSDT',
 'CHZUSDT',
 'BANDUSDT',
 'BUSDUSDT',
 'BEAMUSDT',
 'XTZUSDT',
 'RENUSDT',
 'RVNUSDT',
 'HBARUSDT',
 'NKNUSDT',
 'STXUSDT',
 'KAVAUSDT',
 'ARPAUSDT',
 'IOTXUSDT',
 'RLCUSDT',
 'CTXCUSDT',
 'BCHUSDT',
 'TROYUSDT',
 'VITEUSDT',
 'FTTUSDT',


## Calcularemos os ganhos de cada cripto dentro de 1 mês

In [58]:
unit = '1d'
limit = 30

dfs = {}

spot_client = Spot()

data = {}
errors = 0

for symbol in symbols[0:int(len(symbols)/2)]:
# for symbol in symbols[0:1]:
    try:
        print(f"Searching symbol: {symbol}")
        result = spot_client.klines(symbol, unit, limit=limit)
        df = pd.DataFrame(result, columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Assset Volume', 'Number Trades', 'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'])
        df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
        df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')
        df[['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Assset Volume', 'Number Trades', 'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Assset Volume', 'Number Trades', 'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume']].apply(pd.to_numeric)
        df.index = df['Close Time']
        data[symbol] = {"df": df, "return_value": df.iloc[-1]["Close"] / df.iloc[0]["Close"], "volume": df["Volume"].mean()}
    except ClientError as exc:
        errors = errors + 1
data

Searching symbol: BTCUSDT
Searching symbol: ETHUSDT
Searching symbol: BNBUSDT
Searching symbol: NEOUSDT
Searching symbol: LTCUSDT
Searching symbol: QTUMUSDT
Searching symbol: ADAUSDT
Searching symbol: XRPUSDT
Searching symbol: EOSUSDT
Searching symbol: TUSDUSDT
Searching symbol: IOTAUSDT
Searching symbol: XLMUSDT
Searching symbol: ONTUSDT
Searching symbol: TRXUSDT
Searching symbol: ETCUSDT
Searching symbol: ICXUSDT
Searching symbol: NULSUSDT
Searching symbol: VETUSDT
Searching symbol: USDCUSDT
Searching symbol: LINKUSDT
Searching symbol: WAVESUSDT
Searching symbol: BTTUSDT
Searching symbol: ONGUSDT
Searching symbol: HOTUSDT
Searching symbol: ZILUSDT
Searching symbol: ZRXUSDT
Searching symbol: FETUSDT
Searching symbol: BATUSDT
Searching symbol: XMRUSDT
Searching symbol: ZECUSDT
Searching symbol: IOSTUSDT
Searching symbol: CELRUSDT
Searching symbol: DASHUSDT
Searching symbol: NANOUSDT
Searching symbol: OMGUSDT
Searching symbol: THETAUSDT
Searching symbol: ENJUSDT
Searching symbol: MITHUS

{'BTCUSDT': {'df':                          Open Time      Open      High       Low     Close  \
  Close Time                                                                   
  2021-11-19 23:59:59.999 2021-11-19  56891.62  58320.00  55600.00  58052.24   
  2021-11-20 23:59:59.999 2021-11-20  58057.10  59845.00  57353.00  59707.51   
  2021-11-21 23:59:59.999 2021-11-21  59707.52  60029.76  58486.65  58622.02   
  2021-11-22 23:59:59.999 2021-11-22  58617.70  59444.00  55610.00  56247.18   
  2021-11-23 23:59:59.999 2021-11-23  56243.83  58009.99  55317.00  57541.27   
  2021-11-24 23:59:59.999 2021-11-24  57541.26  57735.00  55837.00  57138.29   
  2021-11-25 23:59:59.999 2021-11-25  57138.29  59398.90  57000.00  58960.36   
  2021-11-26 23:59:59.999 2021-11-26  58960.37  59150.00  53500.00  53726.53   
  2021-11-27 23:59:59.999 2021-11-27  53723.72  55280.00  53610.00  54721.03   
  2021-11-28 23:59:59.999 2021-11-28  54716.47  57445.05  53256.64  57274.88   
  2021-11-29 23:59:59.9

In [59]:
import pickle

with open('datasource.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Ordenar pelos ganhos

In [74]:
df_gains = pd.DataFrame([(symbol, data[symbol]["return_value"]) for symbol in data], columns = ["Symbol", "Return"])
df_gains = df_gains.sort_values(by=['Return'], ascending=False)

In [76]:
portfolio = df_gains[:10]
portfolio

,Symbol,Return
101,GXSUSDT,2.790543
75,CTXCUSDT,2.605749
116,ADADOWNUSDT,1.861007
16,NULSUSDT,1.829268
103,MDTUSDT,1.780934
100,BTCDOWNUSDT,1.549044
145,LUNAUSDT,1.512161
49,COSUSDT,1.455715
97,HIVEUSDT,1.441252
50,COCOSUSDT,1.386453


In [82]:
symbols_portfolio = portfolio.Symbol.to_list()
symbols_portfolio

['GXSUSDT',
 'CTXCUSDT',
 'ADADOWNUSDT',
 'NULSUSDT',
 'MDTUSDT',
 'BTCDOWNUSDT',
 'LUNAUSDT',
 'COSUSDT',
 'HIVEUSDT',
 'COCOSUSDT']

In [113]:
row = 1
for symbol in symbols_portfolio:
    close = data[symbol]["df"]["Close"]
    
    indicator_sma_2 = SMAIndicator(close=close, window=2, fillna=False).sma_indicator()
    indicator_sma_5 = SMAIndicator(close=close, window=5, fillna=False).sma_indicator()
    indicator_sma_10 = SMAIndicator(close=close, window=10, fillna=False).sma_indicator()
    
    fig = make_subplots(rows=1, cols=1)
    fig.update_layout(height=500, width=1000, title_text=symbol, title_x=0.5)

    fig.add_trace(go.Scatter(
        x=close.index, y=close, name=f"{symbol}"
    ), row=1, col=1)    
    
    fig.add_trace(go.Scatter(
        x=indicator_sma_5.index, y=indicator_sma_2, name='SMA 2 Days'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=indicator_sma_5.index, y=indicator_sma_5, name='SMA 5 Days'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=indicator_sma_5.index, y=indicator_sma_10, name='SMA 10 Days'
    ), row=1, col=1)

    fig.show()

## Moedas que compramos

In [ ]:
symbols_buyed = ["CTXCUSD","NULSUSD", "COSUSD"]